In [1]:
import pickle
import requests
import pandas as pd

In [2]:
def post_prediction(
    model_id: int, 
    description: str, 
    commit: str, 
    predict_date: str, 
    predict: dict
):
    url = "https://api.mosqlimate.org/api/registry/predictions/"
    headers = {"X-UID-Key": "eduardocorrearaujo:b35a9f09-fbd2-4764-9503-d0650e50d7e1"}
    prediction = {
        "model": model_id,
        "description": description,
        "commit": commit,
        "predict_date": predict_date,
        "prediction": predict
    }
    return requests.post(url, json=prediction, headers=headers)

In [3]:
cities = [3106200]

start_geo_to_uf = { 
       42:'SC', 41:'PR', 43:'RS' ,
       31:'MG', 32:'ES', 33:'RJ', 35:'SP', 
       50:'MS', 51:'MT', 52:'GO', 53:'DF', 
       21:'MA', 22:'PI', 23:'CE', 24:'RN', 25:'PB', 26:'PE', 27:'AL', 28:'SE', 29:'BA',
       11:'RO', 12:'AC', 13:'AM', 14:'RR', 15:'PA', 16:'AP', 17:'TO'    
}

RF predictions:

In [35]:
def load_ml(city, label,  ini_date = '2022-01-02', end_date= '2023-06-25', doenca = 'dengue'):

    data_ml = pickle.load(open(f'./models/gbt/predictions/rf_{city}_{doenca}_{label}_predictions.pkl', 'rb'))
    
    df_ml = pd.DataFrame()

    df_ml['dates'] = data_ml['dates']
    #df_ml['target'] = data_ml['target']
    df_ml['preds'] = data_ml['preds']
    df_ml['lower'] = data_ml['preds25']
    df_ml['upper'] = data_ml['preds975']
    
    df_ml['adm_2'] = city
    
    df_ml['adm_1'] = start_geo_to_uf[int(str(city)[:2])]
    
    df_ml['adm_0'] = 'BR'
    
    #df_ml.dates = pd.to_datetime(df_ml.dates)

    df_ml = df_ml.loc[(df_ml.dates >= ini_date) & (df_ml.dates <= end_date)]
    
    #df_ml.set_index('dates', inplace = True
    
    return df_ml


In [40]:
for c in cities: 
    
    df =  load_ml(city = c, label = 'cluster')
    
    df.dates = df.dates.astype(str)
    
    df = df.reset_index()
    
    del df['index']
    
    model_id = 5
    description= f'Forecast de novos casos para o geocode {c} entre 2022-01-01 e 2023-01-01 usando apenas os dados de todos as cidades clusterizadas com {c} como input'
    commit ='d2d0a675c051c23dfad604775238cad58bbfe92f' 
    predict_date = '2023-09-14'
    predict = df.to_json(orient = 'records', date_format = 'iso')
    
    res = post_prediction(
    model_id, 
    description, 
    commit, 
    predict_date, 
    predict
    )
    

In [41]:
df

,dates,preds,lower,upper,adm_2,adm_1,adm_0
0,2022-01-02,285.45,0.0,2503.790238,3106200,MG,BR
1,2022-01-09,251.66,0.0,2371.116917,3106200,MG,BR
2,2022-01-16,292.90,0.0,2384.109339,3106200,MG,BR
3,2022-01-23,242.63,0.0,2386.049698,3106200,MG,BR
4,2022-01-30,287.60,0.0,2836.654387,3106200,MG,BR
...,...,...,...,...,...,...,...
73,2023-05-28,3197.04,0.0,7286.105417,3106200,MG,BR
74,2023-06-04,1722.93,0.0,5724.272326,3106200,MG,BR
75,2023-06-11,1156.91,0.0,4357.666818,3106200,MG,BR
76,2023-06-18,1016.49,0.0,3621.847394,3106200,MG,BR


In [42]:
df.dtypes

dates     object
preds    float64
lower    float64
upper    float64
adm_2      int64
adm_1     object
adm_0     object
dtype: object

In [43]:
df.loc[df.preds.isna() == True]

,dates,preds,lower,upper,adm_2,adm_1,adm_0


In [44]:
res.text

'{"id": 74, "model": {"id": 5, "name": "Random Forest model with uncertainty computed with conformal prediction", "description": "In this repo, in the path /models/gbt it\'s implemented a random forest regressor model that \\r\\nbased on the last four weeks of data (cases and climate variables), compute the cases in the fourth week ahead.\\r\\n\\r\\nThe predictions for multiple times are obtained in a rolling window fashion, i.e., \\r\\nthe historical data window is moved forward one week at a time, predicting the next fourth week at each step.\\r\\n\\r\\nThe confidence interval of the predictions are computed using the conformal prediction.", "author": {"user": {"name": "Eduardo Correa Araujo", "username": "eduardocorrearaujo"}, "institution": null}, "repository": "https://github.com/eduardocorrearaujo/forecast_models_dengue.git", "implementation_language": {"language": "Python"}, "disease": "dengue", "type": "time", "ADM_level": 2, "time_resolution": "week"}, "description": "Forecast

In [8]:
for c in cities: 
    
    df =  load_ml(city = c, label = 'single')
    
    model_id = 5
    description= f'Forecast de novos casos para o geocode {c} entre 2022-01-01 e 2023-01-01 usando apenas os dados de {c} como input'
    commit ='d2d0a675c051c23dfad604775238cad58bbfe92f' 
    predict_date = '2023-09-14'
    predict = df.to_json(orient = 'records', date_format = 'iso')
    
    res = post_prediction(
    model_id, 
    description, 
    commit, 
    predict_date, 
    predict
    )
    

,dates,preds,lower,upper,adm_2,adm_1,adm_0
616,2022-01-02,40.58,0.0,88.840328,2611606,PE,BR
617,2022-01-09,29.10,0.0,90.540991,2611606,PE,BR
618,2022-01-16,41.15,0.0,169.592928,2611606,PE,BR
619,2022-01-23,78.58,0.0,260.692885,2611606,PE,BR
620,2022-01-30,45.30,0.0,313.233028,2611606,PE,BR


Saídas do modelo de deep learning 

In [13]:
from datetime import datetime

In [14]:
def load_nn(city, label, ini_date = '2022-01-02', end_date= '2023-07-02', doenca = 'dengue'):
    
    data_nn = pickle.load(open(f'./models/neuralnetworks/predictions/lstm_{city}_{doenca}_{label}.pkl', 'rb'))
    
    
    df_nn = pd.DataFrame()

    ini_index = data_nn['indice'].index(datetime.strptime(ini_date, '%Y-%m-%d'))
    end_index = data_nn['indice'].index(datetime.strptime(end_date, '%Y-%m-%d')) + 1 

    df_nn['dates'] = data_nn['indice'][ini_index:end_index]
    #df_nn['target'] = data_nn['target'][ini_index - 7: end_index - 7, -1] * data_nn['factor']
    df_nn['preds']  = (data_nn['pred'].iloc[ini_index - 7: end_index - 7,-1] * data_nn['factor']).values
    df_nn['lower']  = (data_nn['lb'].iloc[ini_index - 7: end_index - 7,-1] * data_nn['factor']).values
    df_nn['upper']  = (data_nn['ub'].iloc[ini_index - 7: end_index - 7,-1] * data_nn['factor']).values
    
    df_nn['adm_2'] = city
    
    df_nn['adm_1'] = start_geo_to_uf[int(str(city)[:2])]
    
    df_nn['adm_0'] = 'BR'
    
    #df_nn.set_index('dates', inplace = True)
    
    #df_nn.index = pd.to_datetime(df_nn.index)

    return df_nn 

In [15]:

for c in cities: 
    
    df = load_nn(city = c, label = 'single')
    
    model_id = 6
    description= f'Forecast de novos casos para o geocode {c} entre 2022-01-01 e 2023-01-01 usando apenas os dados do geocode {c}'
    commit ='14414f63bf3b0c0b8a131577ae50167f0fb17523' 
    predict_date = '2023-09-12'
    predict = df.to_json(orient = 'records', date_format = 'iso')
    
    
    res = post_prediction(
    model_id, 
    description, 
    commit, 
    predict_date, 
    predict
    )


FileNotFoundError: [Errno 2] No such file or directory: './models/neuralnetworks/predictions/lstm_3106200_dengue_single.pkl'

In [45]:
for c in cities: 
    
    df = load_nn(city = c, label = 'cluster')
    df.dates = df.dates.astype(str)
    model_id = 6
    description= f'Forecast de novos casos para o geocode {c} entre 2022-01-01 e 2023-01-01 usando apenas os dados do geocode e das cidades clusterizadas com ele'
    commit ='14414f63bf3b0c0b8a131577ae50167f0fb17523' 
    predict_date = '2023-09-12'
    predict = df.to_json(orient = 'records', date_format = 'iso')
    
    
    res = post_prediction(
    model_id, 
    description, 
    commit, 
    predict_date, 
    predict
    )


In [46]:
res.text

'{"id": 75, "model": {"id": 6, "name": "Deep learning model using BI-LSTM Layers", "description": "In this repo, in the path /models/neuralnetworks it\'s implemented a deep learning model with three\\r\\nlstm layers (the first one is bidirectional) interspersed with dropout layers and a dense layer in the output.\\r\\nThis model computes the number of new cases in the next four weeks\\r\\nbased on the last four weeks of data (cases and climate variables).\\r\\n\\r\\nThe confidence interval of the predictions is computed using dropout and making multiple predictions to compute\\r\\nthe ci of them.", "author": {"user": {"name": "Eduardo Correa Araujo", "username": "eduardocorrearaujo"}, "institution": null}, "repository": "https://github.com/eduardocorrearaujo/forecast_models_dengue.git", "implementation_language": {"language": "Python"}, "disease": "dengue", "type": "time", "ADM_level": 2, "time_resolution": "week"}, "description": "Forecast de novos casos para o geocode 3106200 entre 2